In [109]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import sklearn

In [3]:
sns.set_theme()

## Load and preprocess data

In [16]:
train_data = pd.read_csv('train.csv')
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [20]:
print(train_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [17]:
train_data.shape

(1460, 81)

In [60]:
train_data = pd.read_csv('train.csv')

In [94]:
def preprocess(df : pd.DataFrame) -> pd.DataFrame:
    '''
    Process raw dataframe into one ready for a tensorflow tensor
    '''
    df.drop('Id', inplace=True, axis=1)
    static_cols = df.columns
    for c in static_cols:
        if pd.api.types.is_numeric_dtype(df[c]):
            fill_value = df[c].mean()
            df[c] = df[c].fillna(fill_value)
        elif pd.api.types.is_string_dtype(df[c]):
            one_hot = pd.get_dummies(df[c], prefix=c)
            df.drop(c, inplace=True, axis=1)
            df = df.join(one_hot)

        else:
            raise NotImplementedError('Failed to find contained value of this column')
    return df

In [95]:
train_data = pd.read_csv('train.csv')
train_data = preprocess(train_data)

In [96]:
train_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [97]:
bad_data = train_data[train_data.isna().any(axis=1)]

In [105]:
assert(bad_data.shape[0] == 0) # catch bad preprocess

In [106]:
train_data.shape

(1460, 289)

## Convert to Tensorflow Dataset (and split validation set)

In [110]:

y_df_train = train_data['SalePrice']
x_df_train = train_data.drop('SalePrice', inplace=False, axis=1)

In [112]:
from sklearn.model_selection import train_test_split

X_all_tensor = tf.convert_to_tensor(x_df_train)
Y_all_tensor = tf.convert_to_tensor(y_df_train)



In [113]:
X_train, X_test, y_train, y_test = train_test_split(X_all_tensor,
                                                    Y_all_tensor,
                                                    test_size=0.33,
                                                    random_state=42)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 615,  613, 1303,  486,  561,  308,  461, 1142,  730, 1155, 1203,
        700,  849, 1260,  787,  352,  710,  124,  178,  287, 1407, 1208,
        294,  327, 1456,  841, 1121,  931,  236,   88,  886,  552,  630,
       1352,  665,  900,  290, 1382,  570,  348,  544, 1376,  660, 1111,
        458, 1005,  333,  721, 1289,  678, 1354,  328,  318, 1438,  908,
         12,  171,  260,  778,  818,  759,  138, 1209,  885,  741,  139,
       1276, 1081,  224, 1063,  409,  325,  299, 1242,  705, 1235, 1355,
        903,  829,  503, 1336,    3, 1313,  467,  321,    5, 1239, 1093,
        715,   39,  542,  242,  136,  714,  737, 1301, 1043, 1330, 1103,
        752, 1353,  376,  314,  442,  319,  767,  756,  424,  553, 1035,
        953, 1185,  596,  227, 1106,  771,  933, 1417,   66,  999,  579,
        942,  211, 1148,  745,  536, 1286, 1295,   85, 1314

## Create a model

In [ ]:
def create_non_seq_model():
